# Convert CSV to MongoDB

## Reading the data
We have two CSV files. One containing the amount of currency my simple miner is gathering. Another one containing the worth of my crypto wallet where I bought a few small amounts of crypto currencies.

In [1]:
!ls ../data/

mining.csv wallet.csv


In [2]:
import pandas as pd
pd.__version__

'0.19.2'

### Mining data

In [3]:
# Read data to dataframe
mining_df = pd.read_csv('../data/mining.csv', sep=';')
# Fill missing numbers
mining_df.fillna(0, inplace=True)
mining_df

,Date,DRK,BTC,LTC
0,2017-04-28,0.011367,0,0.000000
1,2017-04-29,0.010086,0,0.000000
2,2017-04-30,0.011435,0,0.000000
3,2017-05-01,0.012280,0,0.000000
4,2017-05-02,0.012998,0,0.000000
5,2017-05-03,0.010910,0,0.000000
6,2017-05-04,0.010627,0.00000446,0.000317
7,2017-05-05,0.007488,0.0005732,0.003707
8,2017-05-06,0.007381,0.0001493,0.008580
9,2017-05-07,0.007243,0.0001518,0.008722


In [4]:
mining_dict = mining_df.to_dict(orient='records')
mining_dict[0]

{'BTC': 0, 'DRK': 0.01136728, 'Date': '2017-04-28', 'LTC': 0.0}

### Wallet data

In [5]:
# Read data to dataframe
wallet_df = pd.read_csv('../data/wallet.csv', sep=';', decimal=",")
# Fill missing numbers
wallet_df.fillna(0, inplace=True)
# Convert types
wallet_df['Time'] = pd.to_datetime(wallet_df['Time'])
wallet_df

,Time,BTC,DOGE,ETH,LTC,REP
0,2017-05-03 23:17:00,19.70,0.00,78.88,20.81,0.00
1,2017-05-04 20:18:00,21.21,0.00,90.45,24.91,0.00
2,2017-05-05 19:11:00,20.10,11.58,91.73,24.06,0.00
3,2017-05-06 18:56:00,20.28,12.37,92.78,25.91,0.00
4,2017-05-09 00:50:00,21.77,20.34,89.27,28.45,0.00
5,2017-05-09 02:15:00,21.99,20.58,88.62,28.49,34.70
6,2017-05-09 23:59:00,22.46,17.51,87.45,30.14,32.45
7,2017-05-11 00:57:00,23.15,18.83,86.94,32.18,34.36
8,2017-05-11 22:17:00,24.17,17.48,87.87,29.62,34.36
9,2017-05-12 01:55:00,24.13,17.99,88.05,30.02,36.08


In [6]:
wallet_dict = wallet_df.to_dict(orient='records')
wallet_dict[0]

{'BTC': 19.7,
 'DOGE': 0.0,
 'ETH': 78.88,
 'LTC': 20.81,
 'REP': 0.0,
 'Time': Timestamp('2017-05-03 23:17:00')}

## Saving the data
### Import the Pymongo library and connect to MongoDB

In [7]:
try:
    import pymongo
except:
    !pip install pymongo
    import pymongo
print(pymongo.version)
client = pymongo.MongoClient('mongodb://localhost:27017/')

3.4.0


### Create a new database for the data

In [8]:
db = client['crypto-data']

### Create a new table for the mining data

In [9]:
minings = db.minings

### Delete the old data

In [10]:
minings.delete_many({})

### Insert the data by looping through the data

In [11]:
for item in mining_dict:
    minings.insert_one(item)

### Check the existing data

In [12]:
minings_in_database = list(minings.find({}))
minings_in_database[0]

{'BTC': 0,
 'DRK': 0.01136728,
 'Date': '2017-04-28',
 'LTC': 0.0,
 '_id': ObjectId('591a222c0a4693e2b6b9755b')}

### Repeat this for the wallet data

In [13]:
wallets = db.wallets
wallets.delete_many({})
for item in wallet_dict:
    wallets.insert_one(item)
wallets_in_database = list(wallets.find({}))
wallets_in_database[0]

{'BTC': 19.7,
 'DOGE': 0.0,
 'ETH': 78.88,
 'LTC': 20.81,
 'REP': 0.0,
 'Time': datetime.datetime(2017, 5, 3, 23, 17),
 '_id': ObjectId('591a222c0a4693e2b6b9756c')}